<a href="https://colab.research.google.com/github/1oneweeek/TIL/blob/main/torch_1%EC%A3%BC%EC%B0%A8%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, ConcatDataset, Dataset

# 1. CustomDataset으로 데이터 load하기

class CustomDataset(Dataset):
    def __init__(self, dataset1, dataset2, transform=None):
        self.dataset1 = dataset1
        self.dataset2 = dataset2
        self.transform = transform

    def __len__(self):
        return len(self.dataset1) + len(self.dataset2)

    def __getitem__(self, idx):
        if idx < len(self.dataset1):
            img, label = self.dataset1[idx]
        else:
            img, label = self.dataset2[idx - len(self.dataset1)]
        if self.transform:
            img = self.transform(img)
        return img, label

# 데이터 전처리: MNIST와 CIFAR 입력 크기 맞춰주기
transform_mnist = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

transform_cifar = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# MNIST, CIFAR-10 데이터셋을 불러온 다음 하나로 합침
trainset_mnist = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform_mnist)
testset_mnist = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform_mnist)

trainset_cifar = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_cifar)
testset_cifar = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_cifar)

trainset = CustomDataset(trainset_mnist, trainset_cifar)
testset = CustomDataset(testset_mnist, testset_cifar)

# DataLoader로 데이터셋을 불러옴
train_loader = DataLoader(trainset, batch_size=64, shuffle=True)
test_loader = DataLoader(testset, batch_size=64, shuffle=False)


# 2. 모델 정의

model = torchvision.models.resnet18(pretrained=False)
model.conv1 = nn.Conv2d(3, 64, kernel_size=(6, 6), stride=(2, 2), padding=(3, 3), bias=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 20)

# 3. loss함수와 SGD 설정

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# 4. 모델 훈련

num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:    # 매 100 미니배치마다 출력
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')

# 5. 모델 평가

correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('모델 정확도 : %d %%' % (
    100 * correct / total))

Files already downloaded and verified
Files already downloaded and verified
[1,   100] loss: 1.247
[1,   200] loss: 0.894
[1,   300] loss: 0.799
[1,   400] loss: 0.757
[1,   500] loss: 0.729
[1,   600] loss: 0.674
[1,   700] loss: 0.675
[1,   800] loss: 0.639
[1,   900] loss: 0.614
[1,  1000] loss: 0.612
[1,  1100] loss: 0.623
[1,  1200] loss: 0.590
[1,  1300] loss: 0.592
[1,  1400] loss: 0.560
[1,  1500] loss: 0.560
[1,  1600] loss: 0.537
[1,  1700] loss: 0.540
[2,   100] loss: 0.485
[2,   200] loss: 0.483
[2,   300] loss: 0.501
[2,   400] loss: 0.487
[2,   500] loss: 0.473
[2,   600] loss: 0.458
[2,   700] loss: 0.461
[2,   800] loss: 0.455
[2,   900] loss: 0.435
[2,  1000] loss: 0.444
[2,  1100] loss: 0.441
[2,  1200] loss: 0.443
[2,  1300] loss: 0.450
[2,  1400] loss: 0.451
[2,  1500] loss: 0.432
[2,  1600] loss: 0.425
[2,  1700] loss: 0.414
[3,   100] loss: 0.360
[3,   200] loss: 0.378
[3,   300] loss: 0.363
[3,   400] loss: 0.364
[3,   500] loss: 0.359
[3,   600] loss: 0.364
[3, 